In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense, Conv2D, Flatten, Dropout, MaxPooling2D,BatchNormalization

import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd


In [2]:
CLASS_NAMES = np.array([])
for i in range(10):
    CLASS_NAMES =np.append(CLASS_NAMES,chr(ord("0")+i))
for i in range(26):
    CLASS_NAMES = np.append(CLASS_NAMES,chr(ord("A")+i))

In [3]:
#no crop function
def parse_label(strIN):
    return (strIN==CLASS_NAMES).astype(float)
def readimg_to_tensor(fn):
    a = tf.io.read_file((fn))
    img = tf.io.decode_jpeg(a)
    img = tf.image.convert_image_dtype(img, dtype=tf.float16)/255.
    img = tf.image.per_image_standardization(img)
    return img
def parseResult(RS):
    for j in range(len(RS[0])):
        PredChr=''
        for i in range(6):
            PredChr+= CLASS_NAMES[(np.where(RS[i][j] == RS[i][j].max())[0][0])]
        if j == 0:
            LB = [PredChr]
        else:
            LB.append(PredChr)      
    return LB

In [4]:
#training data 02
def prepData02_test(): 
  train_dir_data02 = os.path.abspath(os.getcwd())+"/train/data02_train/"
  data2 = pd.read_csv("./train/data02_train.csv")
  for i in range(len(data2)):
      data2.iloc[i,1] = list(data2.iloc[i,1])
  arr = np.zeros([6,50000],str)
  for j in range(6):
      for i in range(len(data2)):
          arr[j][i] = data2.iloc[i,1][j]
  data2 = data2.drop(columns= ["code"]).join(pd.DataFrame(arr.transpose(),columns = ["code0","code1",'code2','code3','code4','code5']))

  #to one hot
  for i in range(6):
      data2.iloc[:,i+1] = data2.iloc[:,i+1].map(parse_label)

  TL = tf.convert_to_tensor(list(data2.iloc[:,1].values))
  TL2 = tf.convert_to_tensor(list(data2.iloc[:,2].values))
  TL3 = tf.convert_to_tensor(list(data2.iloc[:,3].values))
  TL4 = tf.convert_to_tensor(list(data2.iloc[:,4].values))
  TL5 = tf.convert_to_tensor(list(data2.iloc[:,5].values))
  TL6 = tf.convert_to_tensor(list(data2.iloc[:,6].values))

  train_dir2 = tf.data.Dataset.list_files(train_dir_data02+'*.jpg',shuffle=False)
  train_dir2 = train_dir2.shuffle(50000,seed=1)
  test_dir2 = train_dir2.skip(45000)

  train_label2 = tf.data.Dataset.from_tensor_slices((TL,TL2,TL3,TL4,TL5,TL6))
  train_label2 = train_label2.shuffle(50000,seed=1)
  test_label2 = train_label2.skip(45000)

  test_data2 = test_dir2.map(lambda x: readimg_to_tensor(x))
  test_data2 = tf.data.Dataset.zip((test_data2,test_label2))
  test_data2 = test_data2.batch(5000)


  img_test , lb_test = next(iter(test_data2))
  return  img_test, lb_test

In [5]:
img_test, lb_test= prepData02_test()

In [6]:
def allright(inputdata,label):
    '''calculate the accuracy of each digit and of all right'''
    a = np.argmax(inputdata,axis = 2)
    b = np.argmax(lb_test[0],axis=1)
    b = np.expand_dims(b,0)
    for i in range(5):
        b = np.concatenate((b,np.expand_dims(np.argmax(lb_test[i+1],axis=1),0)),axis=0)
    acc_digit = np.mean(a==b,axis=1)
    acc_all = np.mean(np.all(a.transpose()==b.transpose(),axis=1))
    return acc_digit,acc_all 

In [7]:
def allright_ense(inputdata,label):
    '''calculate the accuracy of each digit and of all right'''
    a = inputdata.transpose()
    b = np.argmax(lb_test[0],axis=1)
    b = np.expand_dims(b,0)
    for i in range(5):
        b = np.concatenate((b,np.expand_dims(np.argmax(lb_test[i+1],axis=1),0)),axis=0)
    acc_digit = np.mean(a==b,axis=1)
    acc_all = np.mean(np.all(a.transpose()==b.transpose(),axis=1))
    return acc_digit,acc_all 

# Load model

In [9]:
filepath = "h5s/data02/"

In [31]:
model_paths=["CNN_GRU_4.hdf5","CNN_GRU_5.hdf5"]
#"CNN_GRU_3_fine.hdf5",,"CNN+GRU.hdf5","CNN_GRU_2.hdf5""railway_d2e-1.hdf5","CNN_GRU_3_fine2.hdf5",

In [32]:
def ensemble(model_paths):
    '''return the most probable character index for given models'''
    models = []
    for m in model_paths:
        models.append(tf.keras.models.load_model(filepath+m))
    result = []
    for m in models:
        result.append(m.predict(img_test))
    r = np.array(result)
    for i,m in zip(r,model_paths):
        print('{}   {}'.format(m,allright(i,lb_test)))
    a = np.argmax(r,axis=3)
    a = a.transpose()
    a_p = np.max(r,axis=3)
    a_p = a_p.transpose()

    #return the most probable character index
    vote_outcome=[]
    for j in range(len(a)):
        chr = []
        for i,p in zip(a[j],a_p[j]):
            if(all(i==i[0])):
                chr.append(i[0])
            else:
                chr.append(i[np.argmax(p)])        
        vote_outcome.append(chr)
    vote_outcome = np.array(vote_outcome)
    print(allright_ense(vote_outcome,lb_test))

    #return the most voted character index
    vote_outcome2=[]
    for j in range(len(a)):
        chr = []
        for i,p in zip(a[j],a_p[j]):
            if(all(i==i[0])):
                chr.append(i[0])
            else:
                chr.append(np.bincount(i).argmax())       
        vote_outcome2.append(chr)
    vote_outcome2 = np.array(vote_outcome2)
    print(allright_ense(vote_outcome2,lb_test))
    


In [34]:
def ensemble_chr(model_paths):
    '''return the most probable characters for given models'''
    models = []
    for m in model_paths:
        models.append(tf.keras.models.load_model(filepath+m))
    result = []
    for m in models:
        result.append(m.predict(img_test))
    r = np.array(result)
    a = np.argmax(r,axis=3)
    a = a.transpose()
    a_p = np.max(r,axis=3)
    a_p = a_p.transpose()
    #return the most probable character index
    vote_outcome=[]
    for j in range(len(a)):
        chr = ''
        for i,p in zip(a[j],a_p[j]):
            if(all(i==i[0])):
                chr+=(CLASS_NAMES[i[0]])
            else:
                maj = np.bincount(i).argmax()   #i[np.argmax(p)]
                chr+=(CLASS_NAMES[maj])           
        vote_outcome.append(chr)
    
    return vote_outcome

In [33]:
ensemble(model_paths)

CNN_GRU_4.hdf5   (array([0.994 , 0.9946, 0.9924, 0.996 , 0.9948, 0.9928]), 0.9656)
CNN_GRU_5.hdf5   (array([0.994 , 0.9938, 0.9936, 0.9958, 0.9952, 0.993 ]), 0.9666)
(array([0.9948, 0.9946, 0.9942, 0.997 , 0.9954, 0.9946]), 0.9712)
(array([0.9946, 0.994 , 0.9932, 0.996 , 0.9956, 0.9932]), 0.968)


# 紀錄

## val 02 
1. 97.44%   多數決

model_paths=["CNN_GRU_3_fine.hdf5","railway_d2e-1.hdf5","CNN_GRU_3_fine2.hdf5","CNN_GRU_4.hdf5","CNN_GRU_5.hdf5","CNN+GRU.hdf5","CNN_GRU_2.hdf5"]   

2. 97.46%   多數決

model_paths=["railway_d2e-1.hdf5","CNN_GRU_3_fine2.hdf5","CNN_GRU_4.hdf5","CNN_GRU_5.hdf5","CNN+GRU.hdf5","CNN_GRU_2.hdf5"] 

3. 97.32%    多數決

model_paths=["CNN_GRU_3_fine2.hdf5","CNN_GRU_4.hdf5","CNN_GRU_5.hdf5","CNN+GRU.hdf5","CNN_GRU_2.hdf5"]

4. 97.3%    多數決

model_paths=["CNN_GRU_3_fine.hdf5","CNN_GRU_3_fine2.hdf5","CNN_GRU_4.hdf5","CNN_GRU_5.hdf5","CNN+GRU.hdf5","CNN_GRU_2.hdf5"]